## UW Pharmacy Student Self-Care Analysis (winter3_4)

This notebook is a draft using the "winter3_4" page in the student comment data of the 2022-2023 SY in the UW SoP. The analysis roughly looks at the determinants of the categories of self-care students chose for this quarter using VADER and NLTK sentiment analysis and Pandas table manipulation.

Note: this notebook relies on uploading a single csv sheet for one quarter, along with adding two additional columns called "Category 1" and "Category 2" which categorize each comment based on the 8 facets of self-care.

### Setup

In [1]:
#import python libraries
import numpy as np
import pandas as pd

!pip install vaderSentiment
!pip install --upgrade pip
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

from textblob import TextBlob
import nltk
nltk.download('brown')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

import spacy

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
/shared-libs/python3.9/py/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-06-01 04:37:08.157032: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compi

In [2]:
# Can change the file csv right here
file_csv = "winter3_4.csv"

data = pd.read_csv(file_csv)

first_column = data.columns[1]
data = data[[first_column, "Category 1", "Category 2"]]

In [3]:
all_dups = pd.DataFrame()
for i in range(len(data)):
    if not pd.isnull(data["Category 2"][i]):
        temp = data[[first_column, "Category 2"]].iloc[i]
        all_dups = all_dups.append(temp)
    else:
        continue
all_dups = all_dups[[first_column, "Category 2"]]
all_dups = all_dups.rename(columns={"Category 2": "Category"})

### Working data

In [4]:
data = data.rename(columns={"Category 1": "Category"})
data = data[[first_column, "Category"]]
data = data.append(all_dups)
data

,Win 3_4,Category
0,I spent time before work to do an easy 20-minu...,Physical
1,One of the activities that I chose from the Ch...,Environmental
2,I took a walk around campus with my roommate t...,Physical
3,I worked on more of the Paint by Diamonds.,Mental
4,I met up with my friends for coffee and it was...,Community
...,...,...
89,I took my dog out for a walk today at Woodland...,Physical
92,I took a walk around campus.,Environmental
93,I went out on a walk with my roommate to catch...,Environmental
101,I called a friend whom I haven’t talked to for...,Mental


### Category frequencies within student comments

The comments data is displayed here in terms of each of the 8 self-care categories (physical, mental, community, emotional, environmental, spiritual, and occupational) through a pie chart. 

In [5]:
import plotly.express as px

dfg = data.groupby("Category").count().sort_values(by=first_column, ascending=False)
print("All reflections processed (includes duplicate categories): ", np.sum(dfg[first_column]))

dfg_pie = px.pie(dfg.reset_index(), values='Win 3_4', names='Category', title='Category Frequencies')
dfg_pie

All reflections processed (includes duplicate categories):  146


### Sentiment analysis (positivity polarity score for each comment)

In this section, I scored each of the comments using the VADER sentiment analysis function. The function sentiment_scores(sentence) will return the positivity polarity score for each comment probabilistically between -1 and 1, with -1 being the most negative and 1 being the most positive. Note that this model is untrained and will be a very rough interpretation of the comments, i.e. not trained on what self-care is as a whole but on the general tone of each comment.

In [6]:
# function to print sentiments
# of the sentence (vader)

#taken from geekforgeeks
def sentiment_scores(sentence):
 
    # Create a SentimentIntensityAnalyzer object.
    sid_obj = SentimentIntensityAnalyzer()
 
    # polarity_scores method of SentimentIntensityAnalyzer
    # object gives a sentiment dictionary.
    # which contains pos, neg, neu, and compound scores.
    sentiment_dict = sid_obj.polarity_scores(sentence)

    return sentiment_dict['compound']

In [7]:
scores = pd.DataFrame(columns={"Win 1_2", "Score"})
for i in data[first_column]:
    score = sentiment_scores(i)
    scores = scores.append({first_column: i, "Score": score}, ignore_index=True)

In [8]:
scores_categories = data.merge(scores, on=first_column, how="right")
scores_categories

,Win 3_4,Category,Score,Win 1_2
0,I spent time before work to do an easy 20-minu...,Physical,0.4926,NaN
1,One of the activities that I chose from the Ch...,Environmental,0.9450,NaN
2,One of the activities that I chose from the Ch...,Mental,0.9450,NaN
3,I took a walk around campus with my roommate t...,Physical,0.3182,NaN
4,I worked on more of the Paint by Diamonds.,Mental,0.0000,NaN
...,...,...,...,...
217,I went out on a walk with my roommate to catch...,Environmental,0.3818,NaN
218,I called a friend whom I haven’t talked to for...,Community,0.8481,NaN
219,I called a friend whom I haven’t talked to for...,Mental,0.8481,NaN
220,I went to eat ramen at ramen danbo with my gir...,Physical,0.0000,NaN


In [9]:
scores_categories['Category'].unique()

array(['Physical', 'Environmental', 'Mental', 'Community', 'Emotional',
       'Spiritual'], dtype=object)

In [10]:
fig = px.scatter(scores_categories, y="Score", x="Category", title="Polarity Scores by Category")
mean_scores = scores_categories.groupby('Category').mean()
for c in scores_categories['Category'].unique():
    fig.add_scatter(x=[c],
                    y=[mean_scores.loc[c]['Score']],
                    marker=dict(
                        color='red',
                        size=10
                    ),
                name=f'{c} mean')

fig.show()

In [11]:
px.bar(mean_scores.reset_index().sort_values(by="Score", ascending=False), x="Category", y="Score", title="Average Scores per Category")

### Scores of each category 

In [12]:
print('Physical category table')
scores_categories_physical = scores_categories[scores_categories["Category"] == "Physical"]
scores_categories_physical

Physical category table


,Win 3_4,Category,Score,Win 1_2
0,I spent time before work to do an easy 20-minu...,Physical,0.4926,NaN
3,I took a walk around campus with my roommate t...,Physical,0.3182,NaN
11,I worked out at the IMA.,Physical,0.0000,NaN
14,I love connecting with the environment by enjo...,Physical,0.8885,NaN
17,Took my dog on a walk through my neighborhood.,Physical,0.0000,NaN
...,...,...,...,...
210,"Instead of taking the car, my classmate and I ...",Physical,0.0000,NaN
213,I took my dog out for a walk today at Woodland...,Physical,0.8834,NaN
214,I took a walk around campus.,Physical,0.0000,NaN
216,I went out on a walk with my roommate to catch...,Physical,0.3818,NaN


In [13]:
print('Mental category table')
scores_categories_mental = scores_categories[scores_categories["Category"] == "Mental"]
scores_categories_mental

Mental category table


,Win 3_4,Category,Score,Win 1_2
2,One of the activities that I chose from the Ch...,Mental,0.9450,NaN
4,I worked on more of the Paint by Diamonds.,Mental,0.0000,NaN
8,The activity that resonates with me is Connect...,Mental,0.4939,NaN
9,Given that I have been sick for the past two w...,Mental,0.7269,NaN
12,I started a new Netflix show to balance my stu...,Mental,0.0000,NaN
15,I read a book for the first time in awhile tha...,Mental,0.9912,NaN
19,I took a 20 minutes break having lunch.,Mental,0.0000,NaN
21,This has been a stressful school week with our...,Mental,0.8357,NaN
24,I listened to music for 30 minutes and cooked ...,Mental,0.0000,NaN
26,I listen to my relaxing and slow speed music f...,Mental,0.4939,NaN


In [14]:
print('Community category table')
scores_categories_community = scores_categories[scores_categories["Category"] == "Community"]
scores_categories_community

Community category table


,Win 3_4,Category,Score,Win 1_2
5,I met up with my friends for coffee and it was...,Community,0.9153,NaN
16,Today I had some social interaction with stude...,Community,0.7170,NaN
30,I boxed with people in virtual reality for abo...,Community,0.0000,NaN
37,Organizing the order of priority is a skill th...,Community,0.9866,NaN
39,I watched a movie with my family.,Community,0.0000,NaN
40,I met with friends on Tuesday night. We talked...,Community,0.7269,NaN
54,Something I did this past weekend was get toge...,Community,0.9436,NaN
82,"This week, the activity that resonated with me...",Community,0.7184,NaN
85,This week I will be spending some time express...,Community,0.9796,NaN
96,I called my friend and met my sister for dinner.,Community,0.4939,NaN


In [15]:
print('Emotional category table')
scores_categories_emotional = scores_categories[scores_categories["Category"] == "Emotional"]
scores_categories_emotional

Emotional category table


,Win 3_4,Category,Score,Win 1_2
6,I met up with my friends for coffee and it was...,Emotional,0.9153,NaN
10,I started going to therapy this week! It was j...,Emotional,0.7337,NaN
34,I wrote in my phone journal for 20 minutes.,Emotional,0.0000,NaN
36,I spent an hour speaking to my therapist.,Emotional,0.0000,NaN
38,Organizing the order of priority is a skill th...,Emotional,0.9866,NaN
149,I met up with my friends for coffee and it was...,Emotional,0.9153,NaN
161,Organizing the order of priority is a skill th...,Emotional,0.9866,NaN


In [16]:
print('Spiritual category table')
scores_categories_spiritual = scores_categories[scores_categories["Category"] == "Spiritual"]
scores_categories_spiritual

Spiritual category table


,Win 3_4,Category,Score,Win 1_2
61,I did yoga and meditation for 20 minutes at le...,Spiritual,0.0935,NaN
105,I did yoga in my room and went on a run.,Spiritual,0.0000,NaN
106,I have spent 30 minutes meditating using Sanve...,Spiritual,0.0000,NaN
170,I did yoga and meditation for 20 minutes at le...,Spiritual,0.0935,NaN
203,I did yoga in my room and went on a run.,Spiritual,0.0000,NaN


In [17]:
print('Occupational category table')
scores_categories_occupational = scores_categories[scores_categories["Category"] == "Occupational"]
scores_categories_occupational

Occupational category table


,Win 3_4,Category,Score,Win 1_2


### Frequency of nouns and verbs 

In [18]:
!pip install spacy -q
!python -m spacy download en_core_web_sm -q

from collections import Counter
import en_core_web_sm

2023-06-01 04:37:33.613006: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-01 04:37:33.829351: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-06-01 04:37:33.829569: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-06-01 04:37:33.866847: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-01 04:37:35.375097: W tensorflow/stream_executor/platform/de

Note: Just for the sake of experimentation, I only did this part on comments categorized as "physical." Can easily broaden it to more categories if useful.

This section takes the top 100 ('top' meaning most frequent) words from a particular category along with its count within the comments. Again, note that there may be duplicates depending on if the comment was repeated if it was categorized twice. I then created a function to extract nouns and verbs within that category. This can potentially be helpful if we were to see which action within each category students tended to choose.

In [19]:
def fig_frequencies(tbl, keyword):
    counter = Counter(" ".join(tbl[first_column]).split()).most_common(100)
    array = []
    for c in counter:
        array = np.append(array, c[0])

    sentence = ','.join(array)

    def extract_nouns_verbs(sentence):
        text = nltk.word_tokenize(sentence)
        pos_tagged = nltk.pos_tag(text)
        nouns_verbs = filter(lambda x:x[1]=='NN' or x[1] == 'VB',pos_tagged)
        return list(nouns_verbs)

    nouns_verbs = pd.DataFrame(extract_nouns_verbs(sentence), columns=['Word', 'Word type'])
    counted_words = pd.DataFrame(counter, columns=['Word', 'Word frequency'])

    noun_verb_frequency = nouns_verbs.merge(counted_words, on='Word', how='left').sort_values(by='Word frequency', ascending=False)

    nlp = en_core_web_sm.load()
    def all_nouns(sentence):
        doc = nlp(sentence)
        nouns = [(token.lemma_, "NN") for token in doc if token.pos_ == "NOUN"]
        return nouns

    def all_verbs(sentence):
        doc = nlp(sentence)
        verbs = [(token.lemma_, "VB") for token in doc if token.pos_ == "VERB"]
        return verbs

    nouns_verbs_spacy = all_nouns(sentence) + all_verbs(sentence)

    nouns_verbs_sp = pd.DataFrame(nouns_verbs_spacy, columns=['Word', 'Word type'])
    counted_words = pd.DataFrame(counter, columns=['Word', 'Word frequency'])

    noun_verb_frequency_sp = nouns_verbs_sp.merge(counted_words, on='Word', how='inner').sort_values(by='Word frequency', ascending=False)
    noun_verb_frequency_sp = noun_verb_frequency_sp.drop_duplicates(subset='Word', keep="last")

    fig = px.bar(noun_verb_frequency_sp, x='Word', y='Word frequency', color='Word type', title='Noun/verb frequencies for Category: ' + keyword)
    fig.update_layout(xaxis_categoryorder = 'total descending')

    fig.show()

In [20]:
fig_frequencies(scores_categories_physical, 'Physical');

/shared-libs/python3.9/py/lib/python3.9/site-packages/spacy/language.py:1895: UserWarning:

[W123] Argument disable with value [] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.



In [21]:
fig_frequencies(scores_categories_mental, 'Mental');

In [22]:
fig_frequencies(scores_categories_emotional, 'Emotional');

In [23]:
fig_frequencies(scores_categories_community, 'Community');

In [24]:
fig_frequencies(scores_categories_spiritual, 'Spiritual');

In [26]:
fig_frequencies(scores_categories_occupational, 'Occupational');

/shared-libs/python3.9/py/lib/python3.9/site-packages/spacy/language.py:1895: UserWarning:

[W123] Argument disable with value [] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.



<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=443b2d8b-ed93-43a6-bbe3-c3fe69ac80e1' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>